In [1]:
import tensorflow as tf
import numpy as np

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
data_path = "data/seq2seq/imdb/"
source_file = data_path + "source_id_20.txt"
target_file = data_path + "target_id_20.txt"
vocab_file = data_path + "vocab.txt"
data_path = "data/seq2seq/rumor/"
# source_file = data_path + "source_id_20.txt"
# target_file = data_path + "target_id_20.txt"
source_file = data_path + "source_id.txt"
target_file = data_path + "target_id.txt"
vocab_file = data_path + "vocab.txt"
save_path = "model/save/implementation_4/"
model_file = save_path+"model-10800.meta"
graph_path = "./graphs/implementation_4_graph"
EPOCH_NUM = 1000
BATCH_SIZE = 64
ENCODER_MAX_LENGTH = 3
DECODER_MAX_LENGTH = 20

In [3]:
class Dataloader:
    def __init__(self, batch_size, encoder_max_length, decoder_max_length, vocab_dict):
        self.batch_size = batch_size
        self.encoder_max_length = encoder_max_length
        self.decoder_max_length = decoder_max_length
        self.vocab_dict = vocab_dict
        
    def create_encoder_batches(self, data_file_list):
        """make self.token_stream into a integer stream."""
        self.encoder_token_stream = []
        print("load %s file data.." % ' '.join(data_file_list))
        for data_file in data_file_list:
            with open(data_file, 'r', encoding="utf-8") as f:
                for line in f:
                    line = line.strip()
                    line = line.split()
                    parse_line = [int(x) for x in line]
                    if len(parse_line) < self.encoder_max_length:
                        parse_line.extend([self.vocab_dict['<PAD>']] * (self.encoder_max_length - len(parse_line)))  # padding
                    else:
                        parse_line = parse_line[:self.encoder_max_length]
                    #y.append(self.vocab_dict['<EOS>'])#end token
                    self.encoder_token_stream.append(parse_line)
        self.encoder_num_batch = int(len(self.encoder_token_stream) / self.batch_size)
        # cut the taken_stream's length exactly equal to num_batch * batch_size
        self.encoder_token_stream = self.encoder_token_stream[:self.encoder_num_batch * self.batch_size]
        self.encoder_sequence_batch = np.split(np.array(self.encoder_token_stream), self.encoder_num_batch, 0)
        self.encoder_pointer = 0
        print("      Load %d * %d encoder batches" % (self.encoder_num_batch, self.batch_size))

    def create_decoder_batches(self, data_file_list):
        """make self.token_stream into a integer stream."""
        self.decoder_token_stream = []
        print("load %s file data.." % ' '.join(data_file_list))
        for data_file in data_file_list:
            with open(data_file, 'r', encoding="utf-8") as f:
                for line in f:
                    line = line.strip()
                    line = line.split()
                    parse_line = [int(x) for x in line]
                    if len(parse_line) < self.decoder_max_length:
                        parse_line.extend([self.vocab_dict['<PAD>']] * (self.decoder_max_length - len(parse_line)))  # padding
                    else:
                        parse_line = parse_line[:self.decoder_max_length]
                    #y.append(self.vocab_dict['<EOS>'])#end token
                    self.decoder_token_stream.append(parse_line)
        self.decoder_num_batch = int(len(self.decoder_token_stream) / self.batch_size)
        # cut the taken_stream's length exactly equal to num_batch * batch_size
        self.decoder_token_stream = self.decoder_token_stream[:self.decoder_num_batch * self.batch_size]
        self.decoder_sequence_batch = np.split(np.array(self.decoder_token_stream), self.decoder_num_batch, 0)
        self.decoder_pointer = 0
        print("      Load %d * %d decoder batches" % (self.decoder_num_batch, self.batch_size))    
    
    def next_encoder_batch(self):
        ret = self.encoder_sequence_batch[self.encoder_pointer]
        self.encoder_pointer = (self.encoder_pointer + 1) % (self.encoder_num_batch - 5)
        #x = np.concatenate([np.tile(self.vocab_dict['<GO>'], self.batch_size), ret], axis=0)
        #y = np.concatenate([ret, np.tile(self.vocab_dict['<EOS>'], self.batch_size)], axis=0)
        ret = [np.array(x) for x in ret]
        return np.array(ret)
    
    def next_decoder_batch(self):
        ret = self.decoder_sequence_batch[self.decoder_pointer]
        self.decoder_pointer = (self.decoder_pointer + 1) % (self.decoder_num_batch - 5)
        x = np.column_stack((np.tile(self.vocab_dict['<GO>'], self.batch_size), ret))
        y = np.column_stack((ret, np.tile(self.vocab_dict['<EOS>'], self.batch_size)))
        #x = np.concatenate([np.tile(self.vocab_dict['<GO>'], self.batch_size), ret], axis=0)
        #y = np.concatenate([ret, np.tile(self.vocab_dict['<EOS>'], self.batch_size)], axis=0)
        return x, y

In [4]:
def load_emb_data(emb_dict_file):
    word_dict = {}
    word_list = []
    item = 0
    with open(emb_dict_file, 'r', encoding="utf-8") as f:
        lines = f.readlines()
        for line in lines:
            word = line.strip()
            word_dict[word] = item
            item += 1
            word_list.append(word)
    length = len(word_dict)
    print("Load embedding success! Num: %d" % length)
    return word_dict, length, word_list

In [5]:
if __name__ == "__main__":
    vocab_dict, vocab_size, word_list = load_emb_data(vocab_file)
    dataloader = Dataloader(BATCH_SIZE, ENCODER_MAX_LENGTH, DECODER_MAX_LENGTH,vocab_dict)
    dataloader.create_encoder_batches([source_file])
    with tf.Session() as sess:
        #restore first
        saver = tf.train.import_meta_graph(model_file)
        saver.restore(sess,tf.train.latest_checkpoint(save_path))
        graph = tf.get_default_graph()
        encoder_inputs_placeholder = graph.get_tensor_by_name("placeholder/encoder_inputs:0")
        prediction_op = graph.get_tensor_by_name("train/prediction:0")
        
        for i in range(1):
        #for i in range(dataloader.encoder_num_batch):
            encoder_inputs = dataloader.next_encoder_batch()
            prediction = sess.run([prediction_op], feed_dict={encoder_inputs_placeholder:encoder_inputs})
            for source, target in zip(encoder_inputs, prediction[0]):
                #print(sentence)
                print(" ".join([word_list[x] for x in source if x != vocab_dict["<PAD>"]]), end=" | ")
                print(" ".join([word_list[x] for x in target if (x != vocab_dict["<PAD>"] and x != vocab_dict["<EOS>"])]))

Load embedding success! Num: 11904
load data/seq2seq/rumor/source_id.txt file data..
      Load 217 * 64 encoder batches
INFO:tensorflow:Restoring parameters from model/save/implementation_4/model-10800
刀 砍 李翔 | 曝光 地 <UNK> 李翔 被 <UNK> <UNK>
他 给 单子 | 千万 <UNK> <UNK> 他 给 你 的 单子
机 尾部 水面 | 东经 海域 被 菲律宾 <UNK> 发现 <UNK> 露出 水面 <UNK> <UNK> 乘客 <UNK> 机组人员 聚集 在 <UNK> 机舱 有 数十
事情 真相 水面 | 事情 等 的 事情 <UNK> 大家 <UNK> 有 真相
这 是 发生 | 这 是 一 家 <UNK> 被 灭 这 是 真的
我 还见过 场面 | 我 要 派 场面 <UNK> <UNK> 过 场面
销售 面包 极限 | <UNK> 销售 的 <UNK> 挑战 被 挑战 <UNK> 挑战
包括 居然是 化学 | 国内 的 牙膏 包括 儿童 牙膏 <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> 化学 国内 的 牙膏 包括 <UNK> 儿童
捐款 不是 让 | 我们 的 捐款 让 我 <UNK> <UNK> 多 的 <UNK>
城管 牛逼 反映 | 城管 <UNK> 牛 逼 反映 <UNK> 是 反映
宁继春 看 请 | 联系 人 宁继春 <UNK> 看 一 <UNK> 得 转 的 冷漠 的 人 <UNK> 请 你 伸出 手指 <UNK>
妇幼 保健院 喝 | <UNK> 妇幼 保健院 提示 您 <UNK> <UNK> 妇幼 保健院 提示 您 请 <UNK> 给 孩子 <UNK> <UNK> 妇幼 保健院 提示
政府 还不如 小女孩 | <UNK> 政府 还 说出 傻子 啊
政府 必须取缔 狗肉节 | 政府 <UNK> 派 <UNK> 狗肉节
大家 一起做 努力 | 大家 一起 为 贞子 的 票房 为 为 贞子 努力
举动 拯救 家庭 | 您 的 一个 小 举动 可能 拯救 一个 家庭
我们 捐给 